In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reddit-eli5/reddit_eli5.jsonl
/kaggle/input/chatgpt-detector-training-dataset/processed_dataset.csv


In [3]:
!pip install gensim==4.2.0

  Using cached gensim-4.2.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Can't uninstall 'gensim'. No files were found to uninstall.


In [5]:
import pickle
import math
import gensim.downloader
from tqdm import tqdm 
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [6]:
Corpus = pd.read_csv("/kaggle/input/chatgpt-detector-training-dataset/processed_dataset.csv")
Corpus

,response,label,text_final
0,"['there', 'are', 'many', 'different', 'best', ...",1,"['many', 'different', 'best', 'seller', 'list'..."
1,"['salt', 'is', 'used', 'on', 'roads', 'to', 'h...",1,"['salt', 'use', 'road', 'help', 'melt', 'ice',..."
2,"['there', 'are', 'a', 'few', 'reasons', 'why',...",1,"['reason', 'still', 'sd', 'standard', 'definit..."
3,"['it', 'is', 'generally', 'not', 'acceptable',...",1,"['generally', 'acceptable', 'ethical', 'advoca..."
4,"['after', 'the', 'wright', 'brothers', 'made',...",1,"['wright', 'brother', 'make', 'first', 'powere..."
...,...,...,...
66727,"['your', 'blood', 'contains', 'a', 'clotting',...",0,"['blood', 'contain', 'clotting', 'factor', 'ca..."
66728,"['lot', ""'s"", 'of', 'people', 'without', 'a', ...",0,"['lot', 'people', 'without', 'job', 'unemploym..."
66729,"['gndn', ""'s"", 'joke', 'is', 'very', 'on', 'po...",0,"['gndn', 'joke', 'point', 'question', 'test', ..."
66730,"['the', 'statement', '``', 'the', 'universe', ...",0,"['statement', 'universe', 'infinite', 'mean', ..."


In [7]:
Encoder = LabelEncoder()
Y = Encoder.fit_transform(Corpus['label'])
Y

array([1, 1, 1, ..., 0, 0, 0])

In [8]:
# function to vectorize tokenized text
def generateVectors(X, w2v_model):
    results = np.zeros((len(X), w2v_model.vector_size))
    for i, x in tqdm(enumerate(X)):
        vectors = np.zeros(w2v_model.vector_size)
        for word in x:
            if word in w2v_model:
                vectors = vectors + w2v_model[word]
        results[i] = vectors
    return results

## Try out word2vec

In [10]:
word2vec_google_news = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [11]:
X_w2v_google = generateVectors(Corpus['text_final'], word2vec_google_news)
print(X_w2v_google.shape)
X_w2v_google

66732it [02:10, 512.61it/s]

(66732, 300)


array([[-89.75201416,  64.98712158,  -5.56005859, ..., -14.91723633,
        -59.6159668 ,  92.2800293 ],
       [-87.97949219,  65.46685791,  -9.82751465, ..., -11.0871582 ,
        -61.74951172,  80.47027588],
       [-51.21954346,  44.0201416 ,  -2.15270996, ...,  -8.89550781,
        -43.04516602,  46.13067627],
       ...,
       [-88.29760742,  66.11120605,  -1.62713623, ..., -10.83544922,
        -59.78381348,  84.28887939],
       [-36.22131348,  23.60296631,  -5.54510498, ...,  -5.62878418,
        -22.61218262,  29.33703613],
       [-37.72344971,  28.06030273,  -3.47729492, ...,  -3.62390137,
        -25.07324219,  37.15515137]])

In [12]:
Train_X_w2v_google, Test_X_w2v_google, Train_Y_w2v_google, Test_Y_w2v_google = model_selection.train_test_split(X_w2v_google,Y,test_size=0.3)

In [13]:
%%time
RBF_SVM_w2v = svm.SVC(C=1.0, kernel='rbf')
RBF_SVM_w2v.fit(Train_X_w2v_google,Train_Y_w2v_google)
# predict the labels on validation dataset
predictions_RBF_SVM_w2v = RBF_SVM_w2v.predict(Test_X_w2v_google)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_RBF_SVM_w2v, Test_Y_w2v_google)*100)
print(classification_report(Test_Y_w2v_google, predictions_RBF_SVM_w2v))

SVM Accuracy Score ->  81.90309690309691
              precision    recall  f1-score   support

           0       0.88      0.74      0.80      9972
           1       0.78      0.90      0.83     10048

    accuracy                           0.82     20020
   macro avg       0.83      0.82      0.82     20020
weighted avg       0.83      0.82      0.82     20020

CPU times: user 8min 32s, sys: 213 ms, total: 8min 32s
Wall time: 8min 32s


## Try out FastText

In [14]:
fasttext_wiki = gensim.downloader.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [15]:
X_fasttext_wiki = generateVectors(Corpus['text_final'], fasttext_wiki)
print(X_fasttext_wiki.shape)
X_fasttext_wiki

66732it [03:00, 369.11it/s]

(66732, 300)


array([[ 17.00800951, -30.18607343,  54.24212975, ...,   6.49130541,
        -38.74869294,   8.146639  ],
       [ 13.66908583, -26.25844893,  53.88362481, ...,   5.18313081,
        -41.14974016,  11.88580133],
       [  6.97192101, -17.17145752,  35.20714699, ...,  -1.93044884,
        -26.84190416,   6.10518951],
       ...,
       [ 19.72682297, -23.04983039,  55.67354597, ...,   3.41582947,
        -37.18129034,  14.02315283],
       [  5.81303084, -11.03204946,  21.00760497, ...,   0.92555436,
        -13.76388196,   4.67654148],
       [  4.78334504, -11.10836183,  23.65561442, ...,   0.78942652,
        -16.50244798,   5.84239445]])

In [16]:
Train_X_fasttext, Test_X_fasttext, Train_Y_fasttext, Test_Y_fasttext = model_selection.train_test_split(X_fasttext_wiki,Y,test_size=0.3)

In [17]:
%%time
RBF_SVM_fasttext = svm.SVC(C=1.0, kernel='rbf')
RBF_SVM_fasttext.fit(Train_X_fasttext,Train_Y_fasttext)
# predict the labels on validation dataset
predictions_RBF_SVM_fasttext = RBF_SVM_fasttext.predict(Test_X_fasttext)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_RBF_SVM_fasttext, Test_Y_fasttext)*100)
print(classification_report(Test_Y_fasttext, predictions_RBF_SVM_fasttext))

SVM Accuracy Score ->  81.67332667332667
              precision    recall  f1-score   support

           0       0.88      0.73      0.80      9990
           1       0.77      0.91      0.83     10030

    accuracy                           0.82     20020
   macro avg       0.83      0.82      0.82     20020
weighted avg       0.83      0.82      0.82     20020

CPU times: user 9min 8s, sys: 430 ms, total: 9min 8s
Wall time: 9min 8s


## Try out GloVe

In [18]:
glove_wiki = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [19]:
X_glove_wiki = generateVectors(Corpus['text_final'], glove_wiki)
print(X_glove_wiki.shape)
X_glove_wiki

66732it [03:01, 368.14it/s]

(66732, 300)


array([[-101.13572602,  -27.58322635, -148.18137418, ..., -455.52378311,
        -178.1283407 ,  221.70852152],
       [-117.73828219,  -32.45445761, -132.25610139, ..., -427.47613057,
        -177.22470416,  220.11083141],
       [ -67.73772944,   -2.73593085,  -83.01405598, ..., -278.9922026 ,
         -99.95513196,  149.71877598],
       ...,
       [-111.737603  ,   -9.84714565, -129.75848621, ..., -435.08493644,
        -179.91243315,  210.45346794],
       [ -47.3484421 ,   -9.19572283,  -52.66940638, ..., -149.55064747,
         -59.80283502,   73.56247099],
       [ -52.53902106,  -17.03000472,  -50.33477516, ..., -188.24458209,
         -72.06346846,   85.79869548]])

In [20]:
Train_X_glove, Test_X_glove, Train_Y_glove, Test_Y_glove = model_selection.train_test_split(X_glove_wiki,Y,test_size=0.3)

In [21]:
%%time
RBF_SVM_glove = svm.SVC(C=1.0, kernel='rbf')
RBF_SVM_glove.fit(Train_X_glove,Train_Y_glove)
# predict the labels on validation dataset
predictions_RBF_SVM_glove = RBF_SVM_glove.predict(Test_X_glove)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_RBF_SVM_glove, Test_Y_glove)*100)
print(classification_report(Test_Y_glove, predictions_RBF_SVM_glove))

SVM Accuracy Score ->  82.72727272727273
              precision    recall  f1-score   support

           0       0.87      0.77      0.82     10029
           1       0.79      0.89      0.84      9991

    accuracy                           0.83     20020
   macro avg       0.83      0.83      0.83     20020
weighted avg       0.83      0.83      0.83     20020

CPU times: user 8min 23s, sys: 148 ms, total: 8min 24s
Wall time: 8min 24s
